In [656]:
# BASIC LIBS

# data
import pandas as pd
import numpy as np

import geopandas as gpd
import contextily as ctx
from pyproj import CRS

# stat significance
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
%config InlineBackend.figure_format = 'retina'


# JUPYTER CONFIG
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# helper consts
THOUSAND = 1000
MILLION = 1000000

# DATA

## TRIPS

### load

In [633]:
df = pd.read_csv('./data/georgia_od_data/GDOT_2019_09_large.csv')

In [634]:
df_sample = df

In [635]:
df_sample.columns[1:20]

Index(['month', 'origin_zone_id', 'destination_zone_id', 'origin_zone_name',
       'destination_zone_name', 'origin_state', 'destination_state',
       'origin_flag', 'destination_flag', 'day_type', 'avg_linked_trips',
       'avg_unlinked_trips', 'avg_tours', 'total_linked_trips',
       'total_unlinked_trips', 'total_tours', 'hour_00', 'hour_01', 'hour_02'],
      dtype='object')

### how we define trips

In [636]:
df_sample_ = df_sample.rename({
    "origin_zone_id":"ORIG", 
    "destination_zone_id":"DEST",

}, axis=1)

df_sample_ = df_sample_[["ORIG", "DEST", "destination_state", "total_linked_trips", "total_unlinked_trips",'purpose_hbw', 'purpose_hbo', 'purpose_wbo', 'purpose_obo']]
df_sample_["TRIPS_HOMEBASED"] = df_sample_.purpose_hbw + df_sample_.purpose_hbo
df_sample_ = df_sample_[["ORIG", "DEST", "TRIPS_HOMEBASED", "destination_state"]]

df_sample_["ORIG"] = df_sample_["ORIG"].astype(str)
df_sample_["DEST"] = df_sample_["DEST"].astype(str)

In [637]:
df_sample_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3736797 entries, 0 to 3736796
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   ORIG               object
 1   DEST               object
 2   TRIPS_HOMEBASED    int64 
 3   destination_state  object
dtypes: int64(1), object(3)
memory usage: 114.0+ MB


### POTENTIAL SITES SET

In [638]:
# import random

# POTENTIAL_SITES_NUMBER = 15

# random.seed(41)
# t = random.sample(df_sample_.query("TRIPS_HOMEBASED > 1000").ORIG.unique().tolist(), POTENTIAL_SITES_NUMBER)
# P = set(t)
# P = set(map(str, P))  # Ensure P is also strings

In [639]:
incoming = df_sample_.groupby(['DEST', 'destination_state']).agg({"TRIPS_HOMEBASED":"sum"}).reset_index()

In [640]:
outcoming = df_sample_.groupby('ORIG').agg({"TRIPS_HOMEBASED":"sum"}).reset_index()

In [641]:
df_area_traffic_volume = incoming.merge(outcoming, left_on="DEST", right_on="ORIG", how = "left").dropna()

In [642]:
df_area_traffic_volume['traffic_volume'] = df_area_traffic_volume['TRIPS_HOMEBASED_x'] + df_area_traffic_volume['TRIPS_HOMEBASED_y']

In [643]:
df_area_traffic_volume = df_area_traffic_volume.drop(columns = ['ORIG']).rename({'DEST':'area', 'TRIPS_HOMEBASED_x':'incoming', 'TRIPS_HOMEBASED_y':'outcoming'}, axis = 1)

In [644]:
df_area_traffic_volume = (
df_area_traffic_volume
    .query("destination_state == 'GA'")
    .sort_values('traffic_volume', ascending=False)
)

In [645]:
df_area_traffic_volume.traffic_volume.quantile(0.9)

311247.0

In [646]:
NUMBER_OF_AREAS = df_area_traffic_volume.area.nunique()
POTENTIAL_SITES_NUMBER = int(NUMBER_OF_AREAS * 0.1)

print(NUMBER_OF_AREAS)
print('Number_of_potential sites to pick: ', POTENTIAL_SITES_NUMBER)

7441
Number_of_potential sites to pick:  744


In [647]:
P = set(df_area_traffic_volume.head(POTENTIAL_SITES_NUMBER).area)

### bi-directional -> one directional

In [648]:

"""
Optimized function to calculate trips dictionary for a subset of origins (P),
including only trips that either begin or end at P.

Args:
    df (pd.DataFrame): DataFrame with columns ["ORIG", "DEST", "TRIPS_HOMEBASED"].
    P (set): Subset of ORIG values.
    
Returns:
    dict: Dictionary with keys (P, ORIG) and values as the sum of P -> ORIG and ORIG -> P trips.
"""
df = df_sample_

# Filter the DataFrame to include only rows where ORIG or DEST is in P
df_filtered = df[(df["ORIG"].isin(P)) | (df["DEST"].isin(P))]

# Group by (ORIG, DEST) and sum TRIPS_HOMEBASED
grouped = df_filtered.groupby(["ORIG", "DEST"])["TRIPS_HOMEBASED"].sum()

# Convert grouped data to a dictionary for fast lookups
trip_lookup = grouped.to_dict()

# Initialize the trips dictionary
trips_dict = {}

# Calculate the sum of P -> ORIG and ORIG -> P for all pairs
for p in P:
    for orig in df_filtered["ORIG"].unique():
        # Get P -> ORIG
        p_to_orig = trip_lookup.get((p, orig), 0)
        
        # Get ORIG -> P
        orig_to_p = trip_lookup.get((orig, p), 0)
        
        # Update the dictionary with the sum
        trips_dict[(p, orig)] = p_to_orig + orig_to_p
    

# Keep only non-zero values in trips_dict
trips_dict = {key: value for key, value in trips_dict.items() if value != 0}


In [649]:
# trips_dict now is P -> A = P -> A + A -> P
trips_dict
1

1

## AREAS SET

In [306]:
A = set(df_sample_.ORIG.unique()) | set(df_sample_.DEST.unique())

In [307]:
len(A)

8043

## AREAS POPULATION

### infer from tracts

In [498]:
gdf_block_groups_shapefiles = gpd.read_file(r"./data/block_groups_shapefiles_2019/tl_2019_13_bg.shp")
df_population_by_tract = pd.read_csv('./data/population_by_tract_2021.csv')

# run only with df load, since it shortens str
df_population_by_tract = df_population_by_tract.rename({"Geographic ID for geographic unit":"GEOID", "# Total population, 2021":"POP"}, axis=1)
gdf_block_groups_shapefiles["GEOID"] = gdf_block_groups_shapefiles["GEOID"].str[:-1]

In [499]:
df_join = (
    gdf_block_groups_shapefiles
    [["GEOID", "BLKGRPCE", "geometry"]]
    .merge(df_population_by_tract[["GEOID", "POP"]].astype(str), on = "GEOID", how = "left")
)
df_join.POP.isna().value_counts()

POP
False    3155
True     2378
Name: count, dtype: int64

Tract of 40% of block groups is missing!!!  
reason - different year. tracts have changes from 2019 to 2021  
fix:  

In [526]:
gdf_population_by_tract = (
    gdf_tracts_shapefiles_2021
    [["GEOID", "geometry"]]
    .merge(df_population_by_tract[["GEOID", "POP"]].astype(str), on = "GEOID", how = "right")
)
gdf_population_by_tract.POP.astype(int).sum()

10625615

for each block group, assign population of its corresponding tract (block group data have the format of: tract_id, block_group_number, geometry of blockgroup shape; tract data have geomtetry of tract shape) if are of block group coincide well enough with tract area (in many cases it wont coincide perfectly cuz tract shapes have changed (they are from a different year)

write me a code to do it

p.s. No need to infer block group population in a smart way, just assign the tract population (i will transform it to block group population later)

In [514]:
# Load the shapefiles
block_groups_2019 = gdf_block_groups_shapefiles
tracts_2021 = gdf_population_by_tract

# Ensure both GeoDataFrames have the same CRS
block_groups_2019 = block_groups_2019.to_crs(tracts_2021.crs)


tracts_2021['tract_geom'] = tracts_2021['geometry']
# Spatial join: Assign tract data to block groups
block_groups_with_population = gpd.sjoin(
    block_groups_2019[['GEOID', 'BLKGRPCE', 'geometry']], tracts_2021[['GEOID', 'geometry', 'POP', 'tract_geom']], 
    how='left', 
    predicate='intersects'  # Match tracts and block groups based on overlapping areas
)

# Calculate the percentage overlap for cases where multiple tracts intersect a block group
block_groups_with_population['overlap_area'] = block_groups_with_population.geometry.intersection(
    block_groups_with_population['tract_geom']
).area

# Retain only the tract with the largest overlap for each block group

block_groups_with_population['overlap_percent'] = block_groups_with_population.overlap_area / block_groups_with_population.geometry.area * 100
block_groups_with_population['overlap_percent'] = block_groups_with_population['overlap_percent'].apply(lambda x: round(x, 3))
block_groups_with_population = (
    block_groups_with_population
        .sort_values(['GEOID_left', 'BLKGRPCE', 'overlap_percent'], ascending=[True, True, False])
        .drop_duplicates(subset=['GEOID_left', 'BLKGRPCE'])
)


block_groups_with_population = block_groups_with_population.drop(columns=['index_right', 'GEOID_right', 'tract_geom']).rename({'POP':'tract_pop', 'GEOID_left': 'GEOID'}, axis = 1)

In [515]:
# INFERRING BLOCK GROUP POPULATION FROM TRACT
temp = block_groups_with_population.groupby('GEOID')['BLKGRPCE'].count().reset_index()
temp.columns = ['GEOID', 'tract_block_group_count']
block_groups_with_population = block_groups_with_population.merge(temp, on = 'GEOID', how = 'left')

block_groups_with_population['block_group_population_inferred'] = block_groups_with_population.tract_pop.astype(int) / block_groups_with_population.tract_block_group_count

In [516]:
block_groups_with_population.tract_pop = block_groups_with_population.tract_pop.astype(int)
gdf_block_groups_with_population = block_groups_with_population

In [517]:
gdf_block_groups_with_population.head()

,GEOID,BLKGRPCE,geometry,tract_pop,overlap_area,overlap_percent,tract_block_group_count,block_group_population_inferred
0,13001950100,1,"POLYGON ((-82.31653 31.85050, -82.31468 31.852...",3421,0.017806,100.000,2,1710.500000
1,13001950100,2,"POLYGON ((-82.43165 31.85615, -82.43158 31.861...",3421,0.013871,100.000,2,1710.500000
2,13001950200,1,"POLYGON ((-82.35209 31.84147, -82.35145 31.841...",2491,0.006946,99.162,3,830.333333
3,13001950200,2,"POLYGON ((-82.45868 31.83810, -82.43136 31.837...",2466,0.004259,99.998,3,822.000000
4,13001950200,3,"POLYGON ((-82.35398 31.78107, -82.35342 31.782...",2491,0.003120,82.786,3,830.333333


### bring populations and trips to one dimension

trips per resident = total trips / total population  
trips generated per area = area population * trips per resident
  
we care about
- proportions with real trips per area
- absolute number match with trips amount (sum of trips **we use in the model** == sum of trips of all areas)

In [690]:
trips_t = []
[trips_t.append(trips) for trips in trips_dict.values()]
TRIPS_TOTAL = sum(trips_t)
print('total trips in millions: ', round(TRIPS_TOTAL / MILLION, 2))

POPULATION_TOTAL = gdf_block_groups_with_population.block_group_population_inferred.sum()
print('total population in millions: ', round(POPULATION_TOTAL / MILLION, 2))

TRIPS_PER_RESIDENT = TRIPS_TOTAL / POPULATION_TOTAL
print('trips per 1 resident in Georgia: ', round(TRIPS_PER_RESIDENT, 2))

gdf_block_groups_with_population['area_demand'] = gdf_block_groups_with_population.block_group_population_inferred * TRIPS_PER_RESIDENT


# reorder columns
gdf_block_groups_with_population = gdf_block_groups_with_population[[
    'GEOID', 'BLKGRPCE', 
    'area_demand', 
    'block_group_population_inferred', 'tract_pop', 'overlap_area',
    'overlap_percent', 'tract_block_group_count', 'geometry']]

total trips in millions:  351.04
total population in millions:  7.65
trips per 1 resident in Georgia:  45.89


In [691]:
print(TRIPS_TOTAL / MILLION)
print(gdf_block_groups_with_population.area_demand.sum() / MILLION)
print('double-check that it matches')

351.039109
351.0391090000001
double-check that it matches


## OTHER

### total population mismatch

In [520]:
gdf_block_groups_with_population.block_group_population_inferred.sum() / 1000000

7.6495505166666655

In [534]:
gdf_block_groups_with_population.drop_duplicates('GEOID').tract_pop.sum() / 1000000

7.683327

In [523]:
df_population_by_tract.POP.sum() / 1000000

10.625615

In [540]:
map_my = gdf_block_groups_with_population.explore(
    column='tract_pop',  # The column to color shapes by
    cmap='YlOrRd',                # Colormap (e.g., Yellow-Orange-Red)
    legend=True,                  # Show a legend
    tiles='CartoDB positron'      # Base map style
)

In [544]:
tracts_2021.POP = tracts_2021.POP.astype(int)
map_baseline = tracts_2021.explore(
    column='POP',  # The column to color shapes by
    cmap='YlOrRd',                # Colormap (e.g., Yellow-Orange-Red)
    legend=True,                  # Show a legend
    tiles='CartoDB positron'      # Base map style
)

In [ ]:
# Save the maps to an HTML file
# map_my.save('map_my.html')
# map_baseline.save('map_baseline.html')

True

Visually overall trends are the same  
=> using my total for analysis  
trends should be the same